#### Genetic Programming: Recursive interpreter
CE310 Evolutionary Computation and Genetic Programming<br>
Reinhold Scherer, Spring 2021, Unit 7

In [1]:
# From Unit 6
# define function set and terminal set as lists (arbitrary selection)
# functions have name and arity
funcSet = [['sin', 1], ['cos', 1], ['add', 2], ['sub', 2], ['mul', 2], ['sub', 2]]
# terminals have arity=0
termSet = ['x', '1', '0', '-1', 'rnd(0,1)']


import random

import typing

def chooseRandomElement(funcSet):
    return funcSet[random.randint(0, len(funcSet)-1)]

# Generate a random expression
def genRndExpr(funcSet, termSet, maxD, method):
    if (maxD == 0) or ((method == 'grow') and (random.random() < len(termSet)/(len(termSet)+len(funcSet)))):
        expr = (chooseRandomElement(termSet))
    else:
        func = chooseRandomElement(funcSet)
        expr = [func[0]]
        for i in range(func[1]):
            expr.append((genRndExpr(funcSet, termSet, maxD-1, method)))
            
    return expr

# test tree generation
print(genRndExpr(funcSet, termSet, 3, 'grow'))
print(genRndExpr(funcSet, termSet, 3, 'full'))

x
['add', ['sub', ['sub', 'rnd(0,1)', 'rnd(0,1)'], ['sub', 'x', 'rnd(0,1)']], ['add', ['sub', 'x', 'x'], ['sin', '0']]]


In [2]:
# Define primite class
class primitive:
    def __init__(self, name: str='', arity: int=0, isMacro: bool=False):
        self.name: str = name
        self.arity: int = arity
        self.isMacro: bool = isMacro
        

def pdiv(x,y) -> float:
    return x/y if y else 1

def IFL0(cond, action1, action2):
    if interp(cond) < 0:
        value = interp(action1)
    else:
        value = interp(action2)
    return value

        
import operator
import random

#Define function set
funcSet =[primitive(name='operator.add', arity=2),
          primitive(name='operator.sub', arity=2),
          primitive(name='operator.mul', arity=2),
          primitive(name='pdiv', arity=2),
          primitive(name='IFL0', arity=3, isMacro=True)]

termSet = [primitive(name='x'),
          primitive(name='1'),
          primitive(name='-1'),
          primitive(name='0'),
          primitive(name='random.uniform(-1,1)')]

# Generate a random expression
def genRndExpr(funcSet, termSet, maxD, method):
    if (maxD == 0) or ((method == 'grow') and (random.random() < len(termSet)/(len(termSet)+len(funcSet)))):
        expr = [chooseRandomElement(termSet)]
    else:
        func = chooseRandomElement(funcSet)
        expr = [func]
        for i in range(func.arity):
            expr.append((genRndExpr(funcSet, termSet, maxD-1, method)))
            
    return expr

def makeAnExpression():
    return genRndExpr(funcSet, termSet, 3, 'full')


expr = genRndExpr(funcSet, termSet, 3, 'full')


def printexpr(expr, x=0):
    print((' '*x) + expr[0].name)
    if len(expr) > 0:
        for i in range(1, expr[0].arity+1):
            printexpr(expr[i], x+6)
    return None

printexpr(expr)

operator.add
      pdiv
            operator.mul
                  -1
                  random.uniform(-1,1)
            pdiv
                  x
                  x
      operator.sub
            IFL0
                  random.uniform(-1,1)
                  1
                  -1
            operator.mul
                  1
                  1


In [3]:
# recursive interpreter
def interp(expr):
    if len(expr) > 1: # if there's more than 1 thing in the expression
        proc = expr[0] # we're going to look at the first thing
        if expr[0].isMacro == False: # if it's not a macro
            # we just interpret its arguments and add them to 'args'
            args = [interp(expr[i]) for i in range(1, proc.arity+1)]
        else: # if it's a macro, we treat it like a macro
            # put the non-evaluated arguments into 'args'
            args = [expr[i] for i in range(1, proc.arity+1)]
        # then we evaluate the procedure with its arguments
        value = eval(proc.name+'(*args)')
    else:
        # if this is just 1 element, we evaluate this element
        value = eval(expr[0].name)
    # we return the computed value
    return value


printexpr(expr)
x=15
print(interp(expr))
expr2 = makeAnExpression()
printexpr(expr2)
print(interp(expr2))
expr3 = makeAnExpression()
printexpr(expr3)
print(interp(expr3))
            

operator.add
      pdiv
            operator.mul
                  -1
                  random.uniform(-1,1)
            pdiv
                  x
                  x
      operator.sub
            IFL0
                  random.uniform(-1,1)
                  1
                  -1
            operator.mul
                  1
                  1
0.9638790126435282
operator.mul
      pdiv
            operator.sub
                  1
                  -1
            operator.sub
                  1
                  1
      operator.mul
            pdiv
                  0
                  0
            IFL0
                  x
                  -1
                  1
1
pdiv
      operator.add
            operator.sub
                  x
                  1
            pdiv
                  random.uniform(-1,1)
                  0
      operator.add
            operator.mul
                  x
                  0
            operator.mul
                  -1
                  random.uni

In [6]:
# Define primite class
class primitive:
    def __init__(self, name='', arity=0, isMacro=False, onInitExec=False):
        self.name = name
        self.arity = arity
        self.isMacro = isMacro
        self.onInitExec = onInitExec
        
#Define function set
funcSet =[primitive(name='operator.add', arity=2),
          primitive(name='operator.sub', arity=2),
          primitive(name='operator.mul', arity=2),
          primitive(name='pdiv', arity=2),
          primitive(name='IFL0', arity=3, isMacro=True)]

termSet = [primitive(name='x'),
          primitive(name='1'),
          primitive(name='-1'),
          primitive(name='0'),
          primitive(name='random.uniform(-1,1)', onInitExec=True)]


import copy

def chooseRandomElement(primSet):
    index = random.randint(0, len(primSet)-1)
    if primSet[index].onInitExec == True:
        tmp = copy.deepcopy(primSet[index])
        tmp.name = str(eval(tmp.name))
    else:
        tmp = primSet[index]
        
    return tmp


expr = genRndExpr(funcSet, termSet, 3, 'full')
printexpr(expr)
print(interp(expr))

IFL0
      operator.mul
            operator.sub
                  0
                  1
            operator.sub
                  x
                  1
      operator.add
            IFL0
                  0
                  x
                  x
            operator.sub
                  x
                  -1
      operator.sub
            operator.mul
                  0.6439633164762657
                  x
            operator.sub
                  x
                  0
31


In [8]:
expr2 = makeAnExpression()
printexpr(expr2)
print(interp(expr2))
print("")
expr3 = makeAnExpression()
printexpr(expr3)
print(interp(expr3))

operator.mul
      IFL0
            IFL0
                  x
                  1
                  -1
            pdiv
                  -0.4202934980891335
                  0.3369735555939526
            operator.sub
                  1
                  1
      operator.mul
            operator.add
                  1
                  0
            operator.add
                  0
                  1
-1.2472595879172788

pdiv
      pdiv
            operator.sub
                  -1
                  -1
            pdiv
                  x
                  -1
      operator.add
            operator.mul
                  -1
                  0
            operator.mul
                  x
                  0
1
